<a href="https://colab.research.google.com/github/Farah14-lab/MachineLearning-BigData/blob/main/ML_BigData_Slide30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Connect Google Drive Untuk Ambil Data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install PySpark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=ad146a8cd7b2bb9978d1076ef7ecea1c2213c962434c5cdc84c94852f6f510d4
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [3]:
# Import Library
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("Movie Lens").getOrCreate()

In [5]:
# Parse String Menjadi Objek Rating
def parseRating(str):
    fields = str.split("::")
    assert len(fields) == 4
    return (int(fields[0]), int(fields[1]), float(fields[2]), int(fields[3]))


In [6]:
# Baca File
raw = spark.read.text("/content/drive/MyDrive/BigData (1)/ML/ratings.dat").rdd.map(lambda x: x[0])
ratings = raw.map(parseRating).toDF(["userId", "movieId", "rating", "timestamp"])
ratings.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|   1193|   5.0|978300760|
|     1|    661|   3.0|978302109|
|     1|    914|   3.0|978301968|
|     1|   3408|   4.0|978300275|
|     1|   2355|   5.0|978824291|
+------+-------+------+---------+
only showing top 5 rows



In [7]:
# Data Training 80% dan Test 20%
training, test = ratings.randomSplit([0.8, 0.2])


In [8]:
# Membuat Model
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)
model.save("mymodel")


In [9]:
# Prediksi Data
predictions = model.transform(test)
mse = predictions.withColumn("diff", col("rating") - col("prediction")).select((col("diff") ** 2).alias("squared_diff")).filter(~col("squared_diff").isNull()).agg({"squared_diff": "sum"}).collect()[0][0]
print("Mean Squared Error:", mse)

predictions.show(10)

Mean Squared Error: nan
+------+-------+------+---------+----------+
|userId|movieId|rating|timestamp|prediction|
+------+-------+------+---------+----------+
|     1|      1|   5.0|978824268| 4.3171163|
|     1|    150|   5.0|978301777|  4.282879|
|     1|    260|   4.0|978300760|  3.923459|
|     1|    914|   3.0|978301968| 4.1090107|
|     1|   1028|   5.0|978301777| 4.3578677|
|     1|   1197|   3.0|978302268| 3.9340935|
|     1|   1270|   5.0|978300055|  4.168401|
|     1|   1545|   4.0|978824139| 3.7702816|
|     1|   1961|   5.0|978301590| 4.4183617|
|     1|   2398|   4.0|978302281| 5.0203776|
+------+-------+------+---------+----------+
only showing top 10 rows



In [10]:
# Menyimpan Hasil Prediksi
predictions.write.format("csv").save("ml-bigdata.csv")